Import and initiate findspark
Then import pyspark

In [1]:
import findspark
findspark.init()
import pyspark

Instatiate SparkSession with Hive support

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python Spark SQL example") \
        .config("spark.sql.warehouse.dir", "hdfs://localhost:9000/user/hive/warehouse") \
        .enableHiveSupport() \
        .getOrCreate()

In [3]:
spark.sql("SELECT * FROM customers").show()

+-----------+----------------+-------------+--------------+----------------+
|customer_id|   customer_name|customer_city|customer_state|customer_zipcode|
+-----------+----------------+-------------+--------------+----------------+
|      11039|     Mary Torres|       Caguas|            PR|             725|
|       5623|      Jose Haley|     Columbus|            OH|           43207|
|       5829|      Mary Smith|      Houston|            TX|           77015|
|       6336|  Richard Maddox|       Caguas|            PR|             725|
|       1708|  Margaret Booth|    Arlington|            TX|           76010|
|      10227|  Mary Henderson|       Caguas|            PR|             725|
|        839|     Lisa Walker|       Caguas|            PR|             725|
|       7604|   Jonathan Hill|      Phoenix|            AZ|           85040|
|       6485|Carolyn Sheppard|Pompano Beach|            FL|           33063|
|       4737|    Mary Mendoza|       Caguas|            PR|             725|

In [4]:
spark.sql("SELECT * FROM products").show()

+-----------+--------------------+------------------+-----------+-------------+----------------+-----------+
|salestxn_id|        product_name|  product_category|category_id|product_price|product_quantity|customer_id|
+-----------+--------------------+------------------+-----------+-------------+----------------+-----------+
|      98660|O'Brien Men's Neo...|           Fishing|         45|        49.98|               2|      12080|
|      97749|O'Brien Men's Neo...|           Fishing|         45|        49.98|               2|      12279|
|     103889|Under Armour Wome...|      Boxing & MMA|         12|        31.99|               4|       3343|
|      22751|O'Brien Men's Neo...|           Fishing|         45|        49.98|               5|      12382|
|     129001|Pelican Sunstream...|           Boating|         47|       199.99|               1|       5032|
|     165110|Nike Men's CJ Eli...|            Cleats|         17|       129.99|               1|       2663|
|     140220|Diamon

In [5]:
cStateCount50 = spark.sql("SELECT customer_state, count(*) as state_count FROM customers GROUP BY customer_state HAVING state_count>=50")

Output of a Hive query will be a DataFrame 

In [6]:
type(cStateCount50)

pyspark.sql.dataframe.DataFrame

In [7]:
cStateCount50.printSchema()

root
 |-- customer_state: string (nullable = true)
 |-- state_count: long (nullable = false)



In [8]:
cStateCount50.show()

+--------------+-----------+
|customer_state|state_count|
+--------------+-----------+
|            CA|        187|
|            NY|         79|
|            TX|         62|
|            PR|        505|
+--------------+-----------+



In [9]:
prd200 = spark.sql("SELECT category_id, product_category, count(*) as prdcount FROM products WHERE product_price>200 GROUP BY category_id, product_category ORDER BY product_category")

In [10]:
type(prd200)

pyspark.sql.dataframe.DataFrame

In [11]:
prd200.show()

+-----------+-------------------+--------+
|category_id|   product_category|prdcount|
+-----------+-------------------+--------+
|         40|        Accessories|       7|
|         16|    As Seen on  TV!|       6|
|          3|Baseball & Softball|       4|
|         42|  Bike & Skate Shop|    1377|
|         47|            Boating|       6|
|          9|   Cardio Equipment|       3|
|         37|        Electronics|       9|
|         34|  Golf Bags & Carts|      10|
|         44| Hunting & Shooting|    1785|
|         30|   Men's Golf Clubs|       5|
|         10|  Strength Training|       2|
|          6|   Tennis & Racquet|       3|
+-----------+-------------------+--------+



In [12]:
prd200.printSchema()

root
 |-- category_id: integer (nullable = true)
 |-- product_category: string (nullable = true)
 |-- prdcount: long (nullable = false)



Write these dataframes into Hive tables after creating a new database

In [ ]:
spark.sql("CREATE DATABASE IF NOT EXISTS retaildb")

In [ ]:
cStateCount50.coalesce(1).write.saveAsTable("retaildb.cStateCount50")

In [ ]:
prd200.coalesce(1).write.saveAsTable("retaildb.prd200")